In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
DATA_DIR = 'drive/My Drive/MLLU/finalProject/data'
MODEL_DIR = 'drive/My Drive/MLLU/finalProject/model/copa'
PREDICTION_DIR = 'drive/My Drive/MLLU/finalProject/model/copa'

In [0]:
!pip install -qU t5

     |████████████████████████████████| 143kB 12.0MB/s 
     |████████████████████████████████| 3.3MB 20.6MB/s 
     |████████████████████████████████| 1.0MB 36.4MB/s 
     |████████████████████████████████| 645kB 45.1MB/s 
     |████████████████████████████████| 8.3MB 36.5MB/s 
     |████████████████████████████████| 61kB 9.8MB/s 
     |████████████████████████████████| 296kB 55.3MB/s 
     |████████████████████████████████| 890kB 53.4MB/s 
     |████████████████████████████████| 3.8MB 50.7MB/s 
     |████████████████████████████████| 421.8MB 29kB/s 
     |████████████████████████████████| 3.9MB 48.3MB/s 
     |████████████████████████████████| 450kB 58.8MB/s 
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: google-colab 1.0.0 has requirement six~=1.12.0, but you'll have six 1.14.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatibl

In [0]:
import functools
import t5
import torch
import transformers
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
# model = t5.models.HfPyTorchModel("t5-base", "/tmp/hft5/", device)
model = t5.models.HfPyTorchModel("t5-base", MODEL_DIR, device)

INFO:absl:Loading from drive/My Drive/MLLU/finalProject/model/copa/model-0.checkpoint


In [0]:
import tensorflow_datasets as tfds

ds = tfds.load(
    "super_glue/copa",
    data_dir=DATA_DIR,
    shuffle_files=True,
    # Download data locally for preprocessing to avoid using GCS space.
    download_and_prepare_kwargs={"download_dir": "./downloads"})
ds["train"] = ds["train"].take(100)
print("A few raw validation examples...")
print(ds)
for ex in tfds.as_numpy(ds["train"].take(11)):
  print(ex)

INFO:absl:Load dataset info from drive/My Drive/MLLU/finalProject/data/super_glue/copa/1.0.2
INFO:absl:Reusing dataset super_glue (drive/My Drive/MLLU/finalProject/data/super_glue/copa/1.0.2)
INFO:absl:Constructing tf.data.Dataset for split None, from drive/My Drive/MLLU/finalProject/data/super_glue/copa/1.0.2


A few raw validation examples...
{'test': <_OptionsDataset shapes: {choice1: (), choice2: (), idx: (), label: (), premise: (), question: ()}, types: {choice1: tf.string, choice2: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string, question: tf.string}>, 'train': <TakeDataset shapes: {choice1: (), choice2: (), idx: (), label: (), premise: (), question: ()}, types: {choice1: tf.string, choice2: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string, question: tf.string}>, 'validation': <_OptionsDataset shapes: {choice1: (), choice2: (), idx: (), label: (), premise: (), question: ()}, types: {choice1: tf.string, choice2: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string, question: tf.string}>}
{'choice1': b'I was hunting for a new apartment.', 'choice2': b'I was moving out of my apartment.', 'idx': 386, 'label': 1, 'premise': b'I packed up my belongings.', 'question': b'cause'}
{'choice1': b'The lid on the mailbox broke.', 'choice2': b'I went on vacation for 

In [0]:
possible_labels = [0,1]

In [0]:
import random
import string
 
def randomString():
   stringLength = random.randint(1,15)
   """Generate a random string of random length """
   letters = string.ascii_lowercase
   return ''.join(random.choice(letters) for i in range(stringLength))
 
label_map = {}
label_set = set()
for i in range(len(possible_labels)):
   label_map[possible_labels[i]] = randomString()
   label_set.add(label_map[possible_labels[i]])
 
assert(len(possible_labels) == len(label_set))
 
print(label_map)


{0: 'kprspoiyastmmn', 1: 'pbflvzqppto'}


Now, we write a preprocess function to convert the examples in the `tf.data.Dataset` into a text-to-text format, with both `inputs` and `targets` fields. The preprocessor also normalizes the text by lowercasing it and removing quotes since the answers are sometimes formatted in odd ways. Finally, we prepend 'trivia question:' to the inputs so that the model knows what task it's trying to solve.

In [0]:
def label_preprocessor(ds):

  def normalize_text(text):
    """Lowercase and remove quotes from a TensorFlow string."""
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text,"'(.*)'", r"\1")
    return text
  
  def to_inputs_and_targets(ex):
    return {
        "inputs": normalize_text(ex["input"]),
        "targets": ex["output"],
        "idx": ex["idx"]
    }
  ds_ = ds.map(to_inputs_and_targets,
               num_parallel_calls=tf.data.experimental.AUTOTUNE)
  return ds_


In [0]:
def copa_extract(ds):
  def extract_io(ex):
    return {
        "input": "copa choice1: "+ex["choice1"]+"  choice2: "+
                  ex["choice2"]+"  premise: "+ex["premise"]+"  question: "+ex["question"],
        "output": "cold" if ex["label"] == 0 else "hot",    # change labels
        "idx": ex["idx"]
    }
  return ds.map(extract_io, num_parallel_calls=tf.data.experimental.AUTOTUNE)

t5.data.TaskRegistry.remove("copa_new")
t5.data.TaskRegistry.add(
    "copa_new",
    # A TfdsTask takes in a TFDS name instead of a tf.data.Dataset function.
    t5.data.TfdsTask,
    tfds_name="super_glue/copa:1.0.2",
    tfds_data_dir=DATA_DIR,
    sentencepiece_model_path=t5.data.DEFAULT_SPM_PATH,
    text_preprocessor=[copa_extract, label_preprocessor],
    postprocess_fn=t5.data.postprocessors.lower_text,
    metric_fns=[t5.evaluation.metrics.accuracy]
)

# Load and print a few examples.
import tensorflow.compat.v1 as tf
cola_task = t5.data.TaskRegistry.get("copa_new")
ds = cola_task.get_dataset(split="train", sequence_length={"inputs": 64, "targets":2})
print("A few preprocessed train examples...")
seq_len_max = 0
input_len_max = 0
for ex in tfds.as_numpy(ds):
  seq_len_max = max(seq_len_max, ex['targets'].shape[0])
  input_len_max = max(input_len_max, ex['inputs'].shape[0])
print('seq_len_max:', seq_len_max)
print('input_len_max:', input_len_max)
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)

INFO:absl:Load dataset info from drive/My Drive/MLLU/finalProject/data/super_glue/copa/1.0.2
INFO:absl:Reusing dataset super_glue (drive/My Drive/MLLU/finalProject/data/super_glue/copa/1.0.2)
INFO:absl:Constructing tf.data.Dataset for split train, from drive/My Drive/MLLU/finalProject/data/super_glue/copa/1.0.2
INFO:absl:Load dataset info from drive/My Drive/MLLU/finalProject/data/super_glue/copa/1.0.2


A few preprocessed train examples...
seq_len_max: 2
input_len_max: 55
{'inputs_plaintext': b'copa choice1: i forgot to eat breakfast.  choice2: i was full from breakfast.  premise: my stomach growled.  question: cause', 'inputs': array([ 7326,     9,  1160,   536,    10,     3,    23, 15687,    12,
           3,  1544,  3688,     5,  1160,   357,    10,     3,    23,
          47,   423,    45,  3688,     5,     3, 17398,    10,    82,
        9883,  1604,  1361,     5,   822,    10,  1137,     1]), 'targets_plaintext': b'cold', 'targets': array([2107,    1]), 'idx': 61}
{'inputs_plaintext': b'copa choice1: he got over it easily.  choice2: he refused to talk about it.  premise: the man went into denial about the tragedy.  question: effect', 'inputs': array([ 7326,     9,  1160,   536,    10,     3,    88,   530,   147,
          34,  1153,     5,  1160,   357,    10,     3,    88, 12191,
          12,  1350,    81,    34,     5,     3, 17398,    10,     8,
         388,   877,   139,  

In [0]:
# # Evaluate the pre-trained checkpoint, before further fine-tuning
# model.eval(
#     "super_glue_copa_v102",
#     sequence_length={"inputs": 64, "targets": 4},
#     batch_size=128,
# )

In [0]:
# Evaluate the pre-trained checkpoint, before further fine-tuning
model.eval(
    "copa_new",
    sequence_length={"inputs": 64, "targets": 2},
    batch_size=128,
)

INFO:absl:Load dataset info from drive/My Drive/MLLU/finalProject/data/super_glue/copa/1.0.2
INFO:absl:Reusing dataset super_glue (drive/My Drive/MLLU/finalProject/data/super_glue/copa/1.0.2)
INFO:absl:Constructing tf.data.Dataset for split validation, from drive/My Drive/MLLU/finalProject/data/super_glue/copa/1.0.2
INFO:absl:eval/copa_new/accuracy at step 0: 0.000


In [0]:
# Run 2000 steps of fine-tuning
model.train(
    mixture_or_task_name="copa_new",
    steps=2000,
    save_steps=200,
    sequence_length={"inputs": 64, "targets": 2},
    split="train",
    batch_size=32,
    optimizer=functools.partial(transformers.AdamW, lr=1e-4),
)

INFO:absl:Load dataset info from drive/My Drive/MLLU/finalProject/data/super_glue/copa/1.0.2
INFO:absl:Reusing dataset super_glue (drive/My Drive/MLLU/finalProject/data/super_glue/copa/1.0.2)
INFO:absl:Constructing tf.data.Dataset for split train, from drive/My Drive/MLLU/finalProject/data/super_glue/copa/1.0.2
INFO:absl:Saving checkpoint for step 0


ValueError: ignored

In [0]:
# Evaluate after fine-tuning
model.eval(
    "copa_new",
    checkpoint_steps="all",
    sequence_length={"inputs": 64, "targets": 2},
    batch_size=32,
)

In [0]:
# evaluate results again
f = open(PREDICTION_DIR + "/validation_eval/copa_new_2000_predictions", "r") 
lines = f.readlines()
f.close()
predicts = [line.rstrip() for line in lines]

f = open(PREDICTION_DIR + "/validation_eval/copa_new_targets", "r") 
lines = f.readlines()
f.close()
targets = [line.rstrip() for line in lines]
print('len:', len(predicts), len(targets))
print('predicts:', predicts)
print('targets:', targets)

In [0]:
score = t5.evaluation.metrics.accuracy(targets, predicts)
print(score)

**END**